## Purpose
To keep the data up-to-date, we need to implement a daily update procedure. Here are steps to achieve this:

1. Load FTP directory listing for Pubmed update files
2. Compare directory listing against local store of previously imported files
    1. This might be tricky - each year the collection name changes, but the files should run through. Ie. files with identical two IDs in two different annual collections should contain the same data.
3. Download new files
4. Parse new file to JSON
5. Perform DB update

In [11]:
import ftplib

pm_ftp_url = 'ftp.ncbi.nlm.nih.gov'
pm_update_dir = 'pubmed/updatefiles'

ftp = ftplib.FTP(pm_ftp_url)
#ftp = ftplib.FTP("ftp://ftp.ncbi.nlm.nih.gov/pubmed/updatefiles")

ftp.login()
ftp.cwd(pm_update_dir)
pm_update_files = []

try:
    pm_update_files = ftp.nlst()
except ftplib.error_perm, resp:
    if str(resp) == "550 No files found":
        print "No files in this directory"
    else:
        print "Other error"

#for f in files:
#    print f

In [9]:
print files[0:10]

['README.txt', 'pubmed18n0929.xml.gz', 'pubmed18n0929.xml.gz.md5', 'pubmed18n0929_stats.html', 'pubmed18n0930.xml.gz', 'pubmed18n0937.xml.gz.md5', 'pubmed18n0930_stats.html', 'pubmed18n0931.xml.gz', 'pubmed18n0931.xml.gz.md5', 'pubmed18n0931_stats.html']


In [37]:
# Generate list of documents to inject into Mongo collection

update_dict = []
pm_fid_lst = []
for update_file in pm_update_files:
    if 'xml' in update_file:
        if 'md5' not in update_file:
            this_id = update_file.split('n')[1].split('.')[0]
            this_collection = update_file.split('n')[0]
            this_dict = {'file_name': update_file, 
                         'file_id': this_id, 
                         'pm_collection': this_collection,
                         'downloaded':0}
            update_dict.append(this_dict)
            pm_fid_lst.append(this_id)
            
print update_dict[1:10]

[{'file_name': 'pubmed18n0930.xml.gz', 'downloaded': 0, 'file_id': '0930', 'pm_collection': 'pubmed18'}, {'file_name': 'pubmed18n0931.xml.gz', 'downloaded': 0, 'file_id': '0931', 'pm_collection': 'pubmed18'}, {'file_name': 'pubmed18n0932.xml.gz', 'downloaded': 0, 'file_id': '0932', 'pm_collection': 'pubmed18'}, {'file_name': 'pubmed18n0933.xml.gz', 'downloaded': 0, 'file_id': '0933', 'pm_collection': 'pubmed18'}, {'file_name': 'pubmed18n0934.xml.gz', 'downloaded': 0, 'file_id': '0934', 'pm_collection': 'pubmed18'}, {'file_name': 'pubmed18n0935.xml.gz', 'downloaded': 0, 'file_id': '0935', 'pm_collection': 'pubmed18'}, {'file_name': 'pubmed18n0936.xml.gz', 'downloaded': 0, 'file_id': '0936', 'pm_collection': 'pubmed18'}, {'file_name': 'pubmed18n0937.xml.gz', 'downloaded': 0, 'file_id': '0937', 'pm_collection': 'pubmed18'}, {'file_name': 'pubmed18n0938.xml.gz', 'downloaded': 0, 'file_id': '0938', 'pm_collection': 'pubmed18'}]


#### Establish mongodb connection

In [38]:
import sys
sys.path.append('../../lib')
sys.path.append('../..')
from MongoSession import MongoSession

mongoSession = MongoSession.fromConnectionString()

db = mongoSession.client.GraphSearch

coll = db.import_pubmed

result = coll.find({"file_id":{"$in":pm_fid_lst}})

retrieved_id = []
for doc in result:
    retrieved_id.append(doc['file_id'])

insert_dict = []
for doc in update_dict:
    if doc['file_id'] not in retrieved_id:
        insert_dict.append(doc)
        
coll.insert_many(insert_dict)
    

06/24/2018 06:53:08 PM [INFO]: MongoSession: Mongo db connection successful


## Process non-downloaded files

In [49]:
# Get list of files that have not been downloaded
import sys, os
sys.path.append('../../lib')
sys.path.append('../..')
from MongoSession import MongoSession
from parse_pm_baseline_xml import PM_baseline_XML

pm_ftp_url = 'ftp.ncbi.nlm.nih.gov'
pm_update_dir = 'pubmed/updatefiles'
local_data_dir = 'downloads'
local_proc_dir = 'processed'

mongoSession = MongoSession.fromConnectionString()

db = mongoSession.client.GraphSearch
coll = db.import_pubmed

result = coll.find({'downloaded':0})

to_download = []
for doc in result[1:10]:
    to_download.append(doc['file_name'])
    
print to_download[1:10]

# Download and process
ftp = ftplib.FTP(pm_ftp_url)
ftp.login()
ftp.cwd(pm_update_dir)

for update_file in to_download[0:1]:
    dl_file = os.path.join(local_data_dir, update_file)
    proc_file = os.path.join(local_proc_dir, update_file+'.csv')
    
    print 'downloading'
    with open(dl_file, 'wb') as f:
        ftp.retrbinary('RETR ' + update_file, f.write)
        
    print 'processing'
    xml_obj = PM_baseline_XML(dl_file, proc_file)
    xml_obj.extract_refs()

06/24/2018 07:13:43 PM [INFO]: MongoSession: Mongo db connection successful


[u'pubmed18n0931.xml.gz', u'pubmed18n0932.xml.gz', u'pubmed18n0933.xml.gz', u'pubmed18n0934.xml.gz', u'pubmed18n0935.xml.gz', u'pubmed18n0936.xml.gz', u'pubmed18n0937.xml.gz', u'pubmed18n0938.xml.gz']
downloading
processing


06/24/2018 07:17:23 PM [WARNING]: No file path specified to store log. Any logging will only be written to console.
06/24/2018 07:17:23 PM [INFO]: Parsed article: 27406060
06/24/2018 07:17:23 PM [INFO]: Parsed article: 27406173
06/24/2018 07:17:23 PM [INFO]: Parsed article: 27406216
06/24/2018 07:17:23 PM [INFO]: Parsed article: 27406248
06/24/2018 07:17:23 PM [INFO]: Parsed article: 27406249
06/24/2018 07:17:23 PM [INFO]: Parsed article: 27406250
06/24/2018 07:17:23 PM [INFO]: Parsed article: 27406289
06/24/2018 07:17:23 PM [INFO]: Parsed article: 27406327
06/24/2018 07:17:23 PM [INFO]: Parsed article: 27406394
06/24/2018 07:17:23 PM [INFO]: Parsed article: 27406569
06/24/2018 07:17:23 PM [INFO]: Parsed article: 27406653
06/24/2018 07:17:23 PM [INFO]: Parsed article: 27406695
06/24/2018 07:17:23 PM [INFO]: Parsed article: 27406873
06/24/2018 07:17:23 PM [INFO]: Parsed article: 27406987
06/24/2018 07:17:23 PM [INFO]: Parsed article: 27406988
06/24/2018 07:17:23 PM [INFO]: Parsed articl

06/24/2018 07:17:25 PM [INFO]: Parsed article: 27418034
06/24/2018 07:17:25 PM [INFO]: Parsed article: 27418099
06/24/2018 07:17:25 PM [INFO]: Parsed article: 27418100
06/24/2018 07:17:25 PM [INFO]: Parsed article: 27418101
06/24/2018 07:17:25 PM [INFO]: Parsed article: 27418163
06/24/2018 07:17:25 PM [INFO]: Parsed article: 27418262
06/24/2018 07:17:25 PM [INFO]: Parsed article: 27418366
06/24/2018 07:17:25 PM [INFO]: Parsed article: 27418384
06/24/2018 07:17:25 PM [INFO]: Parsed article: 27418391
06/24/2018 07:17:25 PM [INFO]: Parsed article: 27418420
06/24/2018 07:17:25 PM [INFO]: Parsed article: 27418421
06/24/2018 07:17:25 PM [INFO]: Parsed article: 27418433
06/24/2018 07:17:25 PM [INFO]: Parsed article: 27418434
06/24/2018 07:17:25 PM [INFO]: Parsed article: 27418438
06/24/2018 07:17:25 PM [INFO]: Parsed article: 27418439
06/24/2018 07:17:25 PM [INFO]: Parsed article: 27418442
06/24/2018 07:17:25 PM [INFO]: Parsed article: 27418456
06/24/2018 07:17:25 PM [INFO]: Parsed article: 2

06/24/2018 07:17:26 PM [INFO]: Parsed article: 27425605
06/24/2018 07:17:26 PM [INFO]: Parsed article: 27425606
06/24/2018 07:17:26 PM [INFO]: Parsed article: 27425607
06/24/2018 07:17:26 PM [INFO]: Parsed article: 27425608
06/24/2018 07:17:26 PM [INFO]: Parsed article: 27425609
06/24/2018 07:17:26 PM [INFO]: Parsed article: 27425610
06/24/2018 07:17:26 PM [INFO]: Parsed article: 27425611
06/24/2018 07:17:26 PM [INFO]: Parsed article: 27425612
06/24/2018 07:17:26 PM [INFO]: Parsed article: 27425613
06/24/2018 07:17:26 PM [INFO]: Parsed article: 27425614
06/24/2018 07:17:26 PM [INFO]: Parsed article: 27425615
06/24/2018 07:17:26 PM [INFO]: Parsed article: 27425616
06/24/2018 07:17:26 PM [INFO]: Parsed article: 27425617
06/24/2018 07:17:26 PM [INFO]: Parsed article: 27425618
06/24/2018 07:17:26 PM [INFO]: Parsed article: 27425619
06/24/2018 07:17:26 PM [INFO]: Parsed article: 27425620
06/24/2018 07:17:26 PM [INFO]: Parsed article: 27425621
06/24/2018 07:17:26 PM [INFO]: Parsed article: 2

...

06/24/2018 07:20:59 PM [INFO]: Parsed article: 28851705
06/24/2018 07:20:59 PM [INFO]: Parsed article: 28851706
06/24/2018 07:20:59 PM [INFO]: Parsed article: 28851707
06/24/2018 07:20:59 PM [INFO]: Parsed article: 28851708
06/24/2018 07:20:59 PM [INFO]: Parsed article: 28851709
06/24/2018 07:20:59 PM [INFO]: Parsed article: 28851710
06/24/2018 07:20:59 PM [INFO]: Parsed article: 28851711
06/24/2018 07:20:59 PM [INFO]: Parsed article: 28851712
06/24/2018 07:20:59 PM [INFO]: Parsed article: 28851721
06/24/2018 07:20:59 PM [INFO]: Parsed article: 28851722
06/24/2018 07:20:59 PM [INFO]: Parsed article: 28851723
06/24/2018 07:20:59 PM [INFO]: Parsed article: 28851724
06/24/2018 07:20:59 PM [INFO]: Parsed article: 28851758
06/24/2018 07:20:59 PM [INFO]: Parsed article: 28852223
06/24/2018 07:20:59 PM [INFO]: Parsed article: 28852809
06/24/2018 07:20:59 PM [INFO]: Parsed article: 28852815
06/24/2018 07:20:59 PM [INFO]: Parsed article: 28852821
06/24/2018 07:20:59 PM [INFO]: Parsed article: 2

06/24/2018 07:21:00 PM [INFO]: Parsed article: 28858066
06/24/2018 07:21:00 PM [INFO]: Parsed article: 28858067
06/24/2018 07:21:00 PM [INFO]: Parsed article: 28858191
06/24/2018 07:21:00 PM [INFO]: Parsed article: 28858271
06/24/2018 07:21:00 PM [INFO]: Parsed article: 28858272
06/24/2018 07:21:00 PM [INFO]: Parsed article: 28858273
06/24/2018 07:21:00 PM [INFO]: Parsed article: 28858274
06/24/2018 07:21:00 PM [INFO]: Parsed article: 28858275
06/24/2018 07:21:00 PM [INFO]: Parsed article: 28858276
06/24/2018 07:21:00 PM [INFO]: Parsed article: 28858277
06/24/2018 07:21:00 PM [INFO]: Parsed article: 28858278
06/24/2018 07:21:00 PM [INFO]: Parsed article: 28858279
06/24/2018 07:21:00 PM [INFO]: Parsed article: 28858280
06/24/2018 07:21:00 PM [INFO]: Parsed article: 28858281
06/24/2018 07:21:00 PM [INFO]: Parsed article: 28858282
06/24/2018 07:21:00 PM [INFO]: Parsed article: 28858283
06/24/2018 07:21:00 PM [INFO]: Parsed article: 28858284
06/24/2018 07:21:00 PM [INFO]: Parsed article: 2

06/24/2018 07:21:01 PM [INFO]: Parsed article: 28864276
06/24/2018 07:21:01 PM [INFO]: Parsed article: 28864294
06/24/2018 07:21:01 PM [INFO]: Parsed article: 28864304
06/24/2018 07:21:01 PM [INFO]: Parsed article: 28864313
06/24/2018 07:21:01 PM [INFO]: Parsed article: 28864364
06/24/2018 07:21:01 PM [INFO]: Parsed article: 28864400
06/24/2018 07:21:01 PM [INFO]: Parsed article: 28864441
06/24/2018 07:21:01 PM [INFO]: Parsed article: 28864443
06/24/2018 07:21:01 PM [INFO]: Parsed article: 28864494
06/24/2018 07:21:01 PM [INFO]: Parsed article: 28864500
06/24/2018 07:21:01 PM [INFO]: Parsed article: 28864541
06/24/2018 07:21:01 PM [INFO]: Parsed article: 28864544
06/24/2018 07:21:01 PM [INFO]: Parsed article: 28864549
06/24/2018 07:21:01 PM [INFO]: Parsed article: 28864596
06/24/2018 07:21:01 PM [INFO]: Parsed article: 28864628
06/24/2018 07:21:01 PM [INFO]: Parsed article: 28864813
06/24/2018 07:21:01 PM [INFO]: Parsed article: 28864819
06/24/2018 07:21:01 PM [INFO]: Parsed article: 2

06/24/2018 07:21:02 PM [INFO]: Parsed article: 28867326
06/24/2018 07:21:02 PM [INFO]: Parsed article: 28867327
06/24/2018 07:21:02 PM [INFO]: Parsed article: 28867328
06/24/2018 07:21:02 PM [INFO]: Parsed article: 28867331
06/24/2018 07:21:02 PM [INFO]: Parsed article: 28867340
06/24/2018 07:21:02 PM [INFO]: Parsed article: 28867349
06/24/2018 07:21:02 PM [INFO]: Parsed article: 28867456
06/24/2018 07:21:02 PM [INFO]: Parsed article: 28867496
06/24/2018 07:21:02 PM [INFO]: Parsed article: 28867502
06/24/2018 07:21:02 PM [INFO]: Parsed article: 28867503
06/24/2018 07:21:02 PM [INFO]: Parsed article: 28867532
06/24/2018 07:21:02 PM [INFO]: Parsed article: 28867600
06/24/2018 07:21:02 PM [INFO]: Parsed article: 28867624
06/24/2018 07:21:02 PM [INFO]: Parsed article: 28867694
06/24/2018 07:21:02 PM [INFO]: Parsed article: 28867695
06/24/2018 07:21:02 PM [INFO]: Parsed article: 28867696
06/24/2018 07:21:02 PM [INFO]: Parsed article: 28867697
06/24/2018 07:21:02 PM [INFO]: Parsed article: 2

06/24/2018 07:21:02 PM [INFO]: Parsed article: 28872479
06/24/2018 07:21:02 PM [INFO]: Parsed article: 28872480
06/24/2018 07:21:02 PM [INFO]: Parsed article: 28872482
06/24/2018 07:21:02 PM [INFO]: Parsed article: 28872484
06/24/2018 07:21:02 PM [INFO]: Parsed article: 28872656
06/24/2018 07:21:02 PM [INFO]: Parsed article: 28872731
06/24/2018 07:21:02 PM [INFO]: Parsed article: 28872779
06/24/2018 07:21:02 PM [INFO]: Parsed article: 28872824
06/24/2018 07:21:02 PM [INFO]: Parsed article: 28872847
06/24/2018 07:21:02 PM [INFO]: Parsed article: 28872889
06/24/2018 07:21:02 PM [INFO]: Parsed article: 28872894
06/24/2018 07:21:02 PM [INFO]: Parsed article: 28872922
06/24/2018 07:21:02 PM [INFO]: Parsed article: 28873046
06/24/2018 07:21:02 PM [INFO]: Parsed article: 28873047


## Debug dataLoader.py for Pubmed articles

In [2]:
import sys, os
sys.path.append('../../lib')
sys.path.append('../..')

from dataLoader import pubmedLoader
from utils.logger import LogHandler
log = LogHandler.get_logger('__name__')

pubmed_loader = pubmedLoader()
log.info('Retrieving FTP files.')
pm_update_files = pubmed_loader.get_ftp_files()
log.info('Generating injection dictionary.')
update_dict, pm_fid_lst = pubmed_loader.generate_injection_dict(pm_update_files)

log.info('Injecting FTP file infos into DB.')
pubmed_loader.inject_new_files(update_dict, pm_fid_lst)
log.info('Downloading and processing new files.')
pubmed_loader.process_new_files()

07/14/2018 02:06:40 PM [INFO]: MongoSession: Mongo db connection successful
07/14/2018 02:06:40 PM [INFO]: Retrieving FTP files.
07/14/2018 02:06:42 PM [INFO]: Generating injection dictionary.
07/14/2018 02:06:42 PM [INFO]: Injecting FTP file infos into DB.
07/14/2018 02:06:42 PM [INFO]: Downloading and processing new files.


[u'pubmed18n0930.xml.gz', u'pubmed18n0931.xml.gz', u'pubmed18n0932.xml.gz', u'pubmed18n0933.xml.gz', u'pubmed18n0934.xml.gz', u'pubmed18n0935.xml.gz', u'pubmed18n0936.xml.gz', u'pubmed18n0937.xml.gz', u'pubmed18n0938.xml.gz']


07/14/2018 02:06:43 PM [INFO]: Downloading file pubmed18n0929.xml.gz
07/14/2018 02:06:53 PM [INFO]: Processing file downloads/pubmed18n0929.xml.gz
07/14/2018 02:09:18 PM [WARNING]: No file path specified to store log. Any logging will only be written to console.
07/14/2018 02:09:18 PM [INFO]: Parsed article: 165
07/14/2018 02:09:18 PM [INFO]: Parsed article: 2396
07/14/2018 02:09:18 PM [INFO]: Parsed article: 2397
07/14/2018 02:09:18 PM [INFO]: Parsed article: 2398
07/14/2018 02:09:18 PM [INFO]: Parsed article: 2399
07/14/2018 02:09:18 PM [INFO]: Parsed article: 2400
07/14/2018 02:09:18 PM [INFO]: Parsed article: 3301
07/14/2018 02:09:18 PM [INFO]: Parsed article: 4243
07/14/2018 02:09:18 PM [INFO]: Parsed article: 4244
07/14/2018 02:09:18 PM [INFO]: Parsed article: 4245
07/14/2018 02:09:18 PM [INFO]: Parsed article: 4246
07/14/2018 02:09:18 PM [INFO]: Parsed article: 4247
07/14/2018 02:09:18 PM [INFO]: Parsed article: 4248
07/14/2018 02:09:18 PM [INFO]: Parsed article: 4249
07/14/2018

07/14/2018 02:09:19 PM [INFO]: Parsed article: 40720
07/14/2018 02:09:19 PM [INFO]: Parsed article: 41651
07/14/2018 02:09:19 PM [INFO]: Parsed article: 41655
07/14/2018 02:09:19 PM [INFO]: Parsed article: 41656
07/14/2018 02:09:19 PM [INFO]: Parsed article: 42496
07/14/2018 02:09:19 PM [INFO]: Parsed article: 43203
07/14/2018 02:09:19 PM [INFO]: Parsed article: 44249
07/14/2018 02:09:19 PM [INFO]: Parsed article: 47276
07/14/2018 02:09:19 PM [INFO]: Parsed article: 47791
07/14/2018 02:09:19 PM [INFO]: Parsed article: 49230
07/14/2018 02:09:19 PM [INFO]: Parsed article: 52420
07/14/2018 02:09:19 PM [INFO]: Parsed article: 52567
07/14/2018 02:09:19 PM [INFO]: Parsed article: 52568
07/14/2018 02:09:19 PM [INFO]: Parsed article: 55319
07/14/2018 02:09:19 PM [INFO]: Parsed article: 55320
07/14/2018 02:09:19 PM [INFO]: Parsed article: 57015
07/14/2018 02:09:19 PM [INFO]: Parsed article: 57016
07/14/2018 02:09:19 PM [INFO]: Parsed article: 59643
07/14/2018 02:09:19 PM [INFO]: Parsed article:

07/14/2018 02:09:20 PM [INFO]: Parsed article: 170019
07/14/2018 02:09:20 PM [INFO]: Parsed article: 170023
07/14/2018 02:09:20 PM [INFO]: Parsed article: 170024
07/14/2018 02:09:20 PM [INFO]: Parsed article: 170025
07/14/2018 02:09:20 PM [INFO]: Parsed article: 170026
07/14/2018 02:09:20 PM [INFO]: Parsed article: 171101
07/14/2018 02:09:20 PM [INFO]: Parsed article: 171106
07/14/2018 02:09:20 PM [INFO]: Parsed article: 172263
07/14/2018 02:09:20 PM [INFO]: Parsed article: 173480
07/14/2018 02:09:20 PM [INFO]: Parsed article: 174838
07/14/2018 02:09:20 PM [INFO]: Parsed article: 174839
07/14/2018 02:09:20 PM [INFO]: Parsed article: 174840
07/14/2018 02:09:20 PM [INFO]: Parsed article: 175981
07/14/2018 02:09:20 PM [INFO]: Parsed article: 175982
07/14/2018 02:09:20 PM [INFO]: Parsed article: 177231
07/14/2018 02:09:20 PM [INFO]: Parsed article: 177232
07/14/2018 02:09:20 PM [INFO]: Parsed article: 177233
07/14/2018 02:09:20 PM [INFO]: Parsed article: 177234
07/14/2018 02:09:20 PM [INFO

...

07/14/2018 02:12:30 PM [INFO]: Parsed article: 27374158
07/14/2018 02:12:30 PM [INFO]: Parsed article: 27374161
07/14/2018 02:12:30 PM [INFO]: Parsed article: 27374162
07/14/2018 02:12:30 PM [INFO]: Parsed article: 27374358
07/14/2018 02:12:30 PM [INFO]: Parsed article: 27374359
07/14/2018 02:12:30 PM [INFO]: Parsed article: 27374360
07/14/2018 02:12:30 PM [INFO]: Parsed article: 27374361
07/14/2018 02:12:30 PM [INFO]: Parsed article: 27374362
07/14/2018 02:12:30 PM [INFO]: Parsed article: 27374363
07/14/2018 02:12:30 PM [INFO]: Parsed article: 27374364
07/14/2018 02:12:30 PM [INFO]: Parsed article: 27374401
07/14/2018 02:12:30 PM [INFO]: Parsed article: 27374623
07/14/2018 02:12:30 PM [INFO]: Parsed article: 27374639
07/14/2018 02:12:30 PM [INFO]: Parsed article: 27374665
07/14/2018 02:12:30 PM [INFO]: Parsed article: 27374775
07/14/2018 02:12:30 PM [INFO]: Parsed article: 27374838
07/14/2018 02:12:30 PM [INFO]: Parsed article: 27374926
07/14/2018 02:12:30 PM [INFO]: Parsed article: 2

07/14/2018 02:12:31 PM [INFO]: Parsed article: 27381223
07/14/2018 02:12:31 PM [INFO]: Parsed article: 27381224
07/14/2018 02:12:31 PM [INFO]: Parsed article: 27381225
07/14/2018 02:12:31 PM [INFO]: Parsed article: 27381226
07/14/2018 02:12:31 PM [INFO]: Parsed article: 27381227
07/14/2018 02:12:31 PM [INFO]: Parsed article: 27381228
07/14/2018 02:12:31 PM [INFO]: Parsed article: 27381327
07/14/2018 02:12:31 PM [INFO]: Parsed article: 27381372
07/14/2018 02:12:31 PM [INFO]: Parsed article: 27381379
07/14/2018 02:12:31 PM [INFO]: Parsed article: 27381466
07/14/2018 02:12:31 PM [INFO]: Parsed article: 27381474
07/14/2018 02:12:31 PM [INFO]: Parsed article: 27381478
07/14/2018 02:12:31 PM [INFO]: Parsed article: 27381567
07/14/2018 02:12:31 PM [INFO]: Parsed article: 27381579
07/14/2018 02:12:31 PM [INFO]: Parsed article: 27381663
07/14/2018 02:12:31 PM [INFO]: Parsed article: 27381681
07/14/2018 02:12:31 PM [INFO]: Parsed article: 27381682
07/14/2018 02:12:31 PM [INFO]: Parsed article: 2

07/14/2018 02:12:32 PM [INFO]: Parsed article: 27385468
07/14/2018 02:12:32 PM [INFO]: Parsed article: 27385528
07/14/2018 02:12:32 PM [INFO]: Parsed article: 27385545
07/14/2018 02:12:32 PM [INFO]: Parsed article: 27385548
07/14/2018 02:12:32 PM [INFO]: Parsed article: 27385549
07/14/2018 02:12:32 PM [INFO]: Parsed article: 27385588
07/14/2018 02:12:32 PM [INFO]: Parsed article: 27385589
07/14/2018 02:12:32 PM [INFO]: Parsed article: 27385647
07/14/2018 02:12:32 PM [INFO]: Parsed article: 27385814
07/14/2018 02:12:32 PM [INFO]: Parsed article: 27385816
07/14/2018 02:12:32 PM [INFO]: Parsed article: 27385817
07/14/2018 02:12:32 PM [INFO]: Parsed article: 27385818
07/14/2018 02:12:32 PM [INFO]: Parsed article: 27385820
07/14/2018 02:12:32 PM [INFO]: Parsed article: 27385821
07/14/2018 02:12:32 PM [INFO]: Parsed article: 27385822
07/14/2018 02:12:32 PM [INFO]: Parsed article: 27385825
07/14/2018 02:12:32 PM [INFO]: Parsed article: 27385827
07/14/2018 02:12:32 PM [INFO]: Parsed article: 2

07/14/2018 02:12:33 PM [INFO]: Parsed article: 27391964
07/14/2018 02:12:33 PM [INFO]: Parsed article: 27392077
07/14/2018 02:12:33 PM [INFO]: Parsed article: 27392081
07/14/2018 02:12:33 PM [INFO]: Parsed article: 27392124
07/14/2018 02:12:33 PM [INFO]: Parsed article: 27392131
07/14/2018 02:12:33 PM [INFO]: Parsed article: 27392196
07/14/2018 02:12:33 PM [INFO]: Parsed article: 27392298
07/14/2018 02:12:33 PM [INFO]: Parsed article: 27392366
07/14/2018 02:12:33 PM [INFO]: Parsed article: 27392483
07/14/2018 02:12:33 PM [INFO]: Parsed article: 27392567
07/14/2018 02:12:33 PM [INFO]: Parsed article: 27392582
07/14/2018 02:12:33 PM [INFO]: Parsed article: 27392601
07/14/2018 02:12:33 PM [INFO]: Parsed article: 27392617
07/14/2018 02:12:33 PM [INFO]: Parsed article: 27392640
07/14/2018 02:12:33 PM [INFO]: Parsed article: 27392641
07/14/2018 02:12:33 PM [INFO]: Parsed article: 27392642
07/14/2018 02:12:33 PM [INFO]: Parsed article: 27392709
07/14/2018 02:12:33 PM [INFO]: Parsed article: 2

07/14/2018 02:12:34 PM [INFO]: Parsed article: 27399974
07/14/2018 02:12:34 PM [INFO]: Parsed article: 27400193
07/14/2018 02:12:34 PM [INFO]: Parsed article: 27400233
07/14/2018 02:12:34 PM [INFO]: Parsed article: 27400654
07/14/2018 02:12:34 PM [INFO]: Parsed article: 27400656
07/14/2018 02:12:34 PM [INFO]: Parsed article: 27400704
07/14/2018 02:12:34 PM [INFO]: Parsed article: 27400705
07/14/2018 02:12:34 PM [INFO]: Parsed article: 27400910
07/14/2018 02:12:34 PM [INFO]: Parsed article: 27400912
07/14/2018 02:12:34 PM [INFO]: Parsed article: 27400975
07/14/2018 02:12:34 PM [INFO]: Parsed article: 27400989
07/14/2018 02:12:34 PM [INFO]: Parsed article: 27400990
07/14/2018 02:12:34 PM [INFO]: Parsed article: 27401030
07/14/2018 02:12:34 PM [INFO]: Parsed article: 27401096
07/14/2018 02:12:34 PM [INFO]: Parsed article: 27401106
07/14/2018 02:12:34 PM [INFO]: Parsed article: 27401108
07/14/2018 02:12:34 PM [INFO]: Parsed article: 27401167
07/14/2018 02:12:34 PM [INFO]: Parsed article: 2